# Supervised Retrieval

In this notebook we use the supervised classification model for a supervised crosslingual information retrieval task.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from src.models.predict_model import MAP_score,feature_selection

## I. Import Data

In this section we import the feature dataframe for the retrieval task.

In [2]:
feature_dataframe=pd.read_json("../data/processed/feature_dataframe.json")
feature_retrieval=pd.read_json("../data/processed/feature_retrieval_reduced_alt.json")

In [3]:
feature_dataframe

,source_id,target_id,number_punctuations_total_difference,number_punctuations_total_difference_relative,number_punctuations_total_difference_normalized,number_words_difference,number_words_difference_relative,number_words_difference_normalized,number_unique_words_difference,number_unique_words_difference_relative,...,pca_embeddding_tf_idf_diff_5,pca_embeddding_average_diff_6,pca_embeddding_tf_idf_diff_6,pca_embeddding_average_diff_7,pca_embeddding_tf_idf_diff_7,pca_embeddding_average_diff_8,pca_embeddding_tf_idf_diff_8,pca_embeddding_average_diff_9,pca_embeddding_tf_idf_diff_9,Translation
0,0,0,10,2.5,-0.184124,-5,-0.116279,0.184124,-3,-0.076923,...,-0.008693,-0.002706,-0.002087,0.002263,0.001772,0.028209,0.004103,-0.068713,-0.011380,1
1,1,1,0,0.0,0.000000,0,0.000000,0.000000,0,0.000000,...,-0.003723,-0.053409,-0.009905,-0.082274,-0.017851,0.013112,-0.000663,-0.092950,-0.015789,1
2,2,2,3,0.0,-0.250000,2,0.285714,0.250000,2,0.285714,...,0.051464,-0.095950,-0.036132,-0.080026,-0.030033,0.004956,-0.001648,-0.014240,-0.000623,1
3,3,3,0,0.0,0.028070,4,0.307692,-0.028070,4,0.307692,...,-0.028189,-0.110666,-0.025530,0.014155,0.003518,0.038873,0.007949,-0.096558,-0.020123,1
4,4,4,0,0.0,-0.012605,-3,-0.187500,0.012605,-2,-0.133333,...,-0.018249,-0.111367,-0.028945,0.124814,0.036022,0.060507,0.017436,-0.094980,-0.022765,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219995,19999,13747,3,3.0,-0.109091,6,0.600000,0.109091,5,0.500000,...,-0.024454,0.014361,0.006951,-0.049811,-0.013581,0.033968,0.006560,-0.077568,-0.020721,0
219996,19999,18387,1,1.0,-0.090909,-1,-0.100000,0.090909,-1,-0.100000,...,-0.059574,0.053338,0.015654,-0.016097,-0.005740,0.056920,0.021571,-0.077724,-0.027039,0
219997,19999,4964,0,0.0,0.019481,3,0.300000,-0.019481,3,0.300000,...,-0.037964,0.074301,0.021780,-0.010230,-0.005652,0.070773,0.016596,-0.060947,-0.018164,0
219998,19999,12897,1,1.0,-0.034091,4,0.400000,0.034091,4,0.400000,...,-0.031841,0.039546,0.013315,0.020479,0.002369,0.088429,0.022763,-0.074700,-0.020741,0


In [4]:
feature_retrieval

,source_id,target_id,number_punctuations_total_difference,number_punctuations_total_difference_relative,number_punctuations_total_difference_normalized,number_words_difference,number_words_difference_relative,number_!_difference,number_!_difference_relative,number_!_difference_normalized,...,pca_embeddding_average_diff_2,pca_embeddding_tf_idf_diff_2,pca_embeddding_average_diff_3,pca_embeddding_average_diff_4,pca_embeddding_average_diff_5,pca_embeddding_average_diff_6,pca_embeddding_average_diff_7,pca_embeddding_average_diff_8,pca_embeddding_average_diff_9,Translation
0,20000,20000,1,0.333333,-0.053913,-3,-0.136364,0,0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,20000,20001,1,0.333333,-0.130000,-10,-0.454545,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,20000,20002,-3,-1.000000,0.120000,-19,-0.863636,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,20000,20003,-3,-1.000000,0.120000,-18,-0.818182,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,20000,20004,-2,-0.666667,-0.005000,-15,-0.681818,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,20099,24995,4,0.000000,-0.137931,19,3.166667,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
499996,20099,24996,3,0.000000,-0.176471,8,1.333333,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
499997,20099,24997,2,0.000000,-0.125000,8,1.333333,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
499998,20099,24998,3,0.000000,-0.093750,23,3.833333,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
list(feature_retrieval.columns)

['source_id',
 'target_id',
 'number_punctuations_total_difference',
 'number_punctuations_total_difference_relative',
 'number_punctuations_total_difference_normalized',
 'number_words_difference',
 'number_words_difference_relative',
 'number_!_difference',
 'number_!_difference_relative',
 'number_!_difference_normalized',
 'number_"_difference',
 'number_"_difference_relative',
 'number_"_difference_normalized',
 'number_#_difference',
 'number_$_difference',
 'number_%_difference',
 'number_%_difference_relative',
 'number_&_difference',
 'number_&_difference_relative',
 "number_'_difference",
 'number_(_difference',
 'number_(_difference_relative',
 'number_(_difference_normalized',
 'number_*_difference',
 'number_*_difference_relative',
 'number_*_difference_normalized',
 'number_+_difference',
 'number_,_difference_normalized',
 'number_-_difference',
 'number_-_difference_relative',
 'number_._difference',
 'number_._difference_normalized',
 'number_/_difference',
 'number_/_

In [8]:
keep_columns=['cosine_similarity_average',
 'euclidean_distance_average',
 'jaccard_translation_source',
 'jaccard_numbers_source',
 'pca_embeddding_average_diff_0',
 'pca_embeddding_tf_idf_diff_0',
 'pca_embeddding_average_diff_1',
 'pca_embeddding_average_diff_2',
 'pca_embeddding_tf_idf_diff_2',
 'pca_embeddding_average_diff_3',
 'pca_embeddding_average_diff_4',
 'pca_embeddding_average_diff_5',
 'pca_embeddding_average_diff_6',
 'pca_embeddding_average_diff_7',
 'pca_embeddding_average_diff_8',
 'pca_embeddding_average_diff_9']
features=[ 'number_punctuations_total_difference',
 'number_punctuations_total_difference_relative',
 'number_punctuations_total_difference_normalized',
 'number_words_difference',
 'number_words_difference_relative',
 'number_!_difference',
 'number_!_difference_relative',
 'number_!_difference_normalized',
 'number_"_difference',
 'number_"_difference_relative',
 'number_"_difference_normalized',
 'number_#_difference',
 'number_$_difference',
 'number_%_difference',
 'number_%_difference_relative',
 'number_&_difference',
 'number_&_difference_relative',
 "number_'_difference",
 'number_(_difference',
 'number_(_difference_relative',
 'number_(_difference_normalized',
 'number_*_difference',
 'number_*_difference_relative',
 'number_*_difference_normalized',
 'number_+_difference',
 'number_,_difference_normalized',
 'number_-_difference',
 'number_-_difference_relative',
 'number_._difference',
 'number_._difference_normalized',
 'number_/_difference',
 'number_/_difference_relative',
 'number_:_difference',
 'number_:_difference_relative',
 'number_:_difference_normalized',
 'number_;_difference',
 'number_;_difference_relative',
 'number_;_difference_normalized',
 'number_<_difference',
 'number_<_difference_relative',
 'number_<_difference_normalized',
 'number_=_difference',
 'number_=_difference_relative',
 'number_=_difference_normalized',
 'number_>_difference',
 'number_>_difference_relative',
 'number_>_difference_normalized',
 'number_?_difference',
 'number_?_difference_relative',
 'number_?_difference_normalized',
 'number_@_difference',
 'number_@_difference_relative',
 'number_@_difference_normalized',
 'number_[_difference',
 'number_\\_difference',
 'number_\\_difference_relative',
 'number_\\_difference_normalized',
 'number_^_difference',
 'number_^_difference_relative',
 'number_^_difference_normalized',
 'number___difference',
 'number___difference_relative',
 'number___difference_normalized',
 'number_`_difference',
 'number_`_difference_relative',
 'number_`_difference_normalized',
 'number_{_difference',
 'number_{_difference_relative',
 'number_{_difference_normalized',
 'number_|_difference',
 'number_|_difference_relative',
 'number_|_difference_normalized',
 'number_}_difference',
 'number_}_difference_relative',
 'number_}_difference_normalized',
 'number_~_difference',
 'number_~_difference_relative',
 'number_~_difference_normalized',
 'number_characters_difference_normalized',
 'characters_avg_difference',
 'characters_avg_difference_normalized',
 'number_ADJ_difference',
 'number_ADJ_difference_relative',
 'number_ADJ_difference_normalized',
 'number_NOUN_difference_relative',
 'number_NOUN_difference_normalized',
 'number_VERB_difference',
 'number_VERB_difference_relative',
 'number_VERB_difference_normalized',
 'number_Pres_difference',
 'number_Pres_difference_relative',
 'number_Pres_difference_normalized',
 'number_Past_difference',
 'number_Past_difference_relative',
 'number_Past_difference_normalized',
 'number__difference',
 'number__difference_relative',
 'number__difference_normalized',
 'score_polarity_difference',
 'score_polarity_difference_relative',
 'score_polarity_difference_normalized',
 'score_subjectivity_difference',
 'score_subjectivity_difference_relative',
 'score_subjectivity_difference_normalized',
 'number_stopwords_difference_relative',
 'number_stopwords_difference_normalized']
    

In [7]:
data_train=feature_dataframe.filter(items=keep_columns)
data_test=feature_retrieval.filter(items=keep_columns)

In [8]:
data_train

,cosine_similarity_average,euclidean_distance_average,jaccard_translation_source,jaccard_numbers_source,pca_embeddding_average_diff_0,pca_embeddding_tf_idf_diff_0,pca_embeddding_average_diff_1,pca_embeddding_average_diff_2,pca_embeddding_tf_idf_diff_2,pca_embeddding_average_diff_3,pca_embeddding_average_diff_4,pca_embeddding_average_diff_5,pca_embeddding_average_diff_6,pca_embeddding_average_diff_7,pca_embeddding_average_diff_8,pca_embeddding_average_diff_9
0,0.893585,0.235780,0.000000,0.0,-0.273821,-0.042289,0.140056,0.016341,0.003404,0.096514,0.021877,-0.050624,-0.002706,0.002263,0.028209,-0.068713
1,0.915635,0.204485,0.357143,1.0,-0.036356,-0.006908,-0.057238,-0.032582,-0.007806,-0.008849,-0.071427,-0.038196,-0.053409,-0.082274,0.013112,-0.092950
2,0.752805,0.404717,0.250000,0.0,-0.167636,-0.057076,0.053921,-0.022822,-0.008628,0.044912,-0.127683,0.134327,-0.095950,-0.080026,0.004956,-0.014240
3,0.866149,0.273860,0.115385,0.0,-0.287463,-0.073297,0.072464,0.005790,0.002972,0.033527,-0.072829,-0.108856,-0.110666,0.014155,0.038873,-0.096558
4,0.877954,0.277976,0.130435,0.0,-0.369688,-0.097627,0.075213,-0.012726,-0.001670,0.071090,-0.019626,-0.059456,-0.111367,0.124814,0.060507,-0.094980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219995,0.796352,0.346970,0.086957,0.0,-0.170317,-0.056327,0.126002,-0.046761,-0.003717,0.108103,-0.003251,-0.071700,0.014361,-0.049811,0.033968,-0.077568
219996,0.787464,0.367720,0.058824,0.0,-0.259787,-0.087571,0.101450,-0.002358,-0.000053,0.029752,-0.051786,-0.174500,0.053338,-0.016097,0.056920,-0.077724
219997,0.790242,0.353839,0.000000,0.0,-0.207701,-0.059167,0.130391,0.016905,0.014412,0.031497,-0.009676,-0.115054,0.074301,-0.010230,0.070773,-0.060947
219998,0.764747,0.373764,0.000000,0.0,-0.244540,-0.077908,0.105429,0.026805,0.012577,0.061414,-0.046875,-0.105910,0.039546,0.020479,0.088429,-0.074700


In [9]:
data_test

,cosine_similarity_average,euclidean_distance_average,jaccard_translation_source,jaccard_numbers_source,pca_embeddding_average_diff_0,pca_embeddding_tf_idf_diff_0,pca_embeddding_average_diff_1,pca_embeddding_average_diff_2,pca_embeddding_tf_idf_diff_2,pca_embeddding_average_diff_3,pca_embeddding_average_diff_4,pca_embeddding_average_diff_5,pca_embeddding_average_diff_6,pca_embeddding_average_diff_7,pca_embeddding_average_diff_8,pca_embeddding_average_diff_9
0,0.874402,0.246901,0.193548,0.0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.683881,0.414094,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.581202,0.547308,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.663935,0.500478,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.634003,0.485720,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.675244,0.488695,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0
499996,0.725018,0.454538,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0
499997,0.594514,0.539619,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0
499998,0.677087,0.481879,0.032258,0.0,0,0,0,0,0,0,0,0,0,0,0,0


## II. Supervised Retrieval

### First iteration after dropping correlated features from analyes

In [10]:
# import pickle 
# file = open("../data/processed/correlated_features.pkl",'rb')
# correlated_features = pickle.load(file)
# file.close()
# correlated_features

In [11]:
# feature_dataframe=feature_dataframe.drop(columns=correlated_features)
# feature_dataframe

In [12]:
# feature_dataframe=feature_dataframe.drop(columns=['word_mover_distance'])

In [14]:
#feature_dataframe.columns.difference(feature_retrieval.columns)

Index(['characters_avg_difference_relative', 'cosine_similarity_tf_idf',
       'euclidean_distance_tf_idf', 'jaccard_translation_target',
       'number_#_difference_normalized', 'number_#_difference_relative',
       'number_$_difference_normalized', 'number_$_difference_relative',
       'number_%_difference_normalized', 'number_&_difference_normalized',
       'number_'_difference_normalized', 'number_'_difference_relative',
       'number_)_difference', 'number_)_difference_normalized',
       'number_)_difference_relative', 'number_+_difference_normalized',
       'number_+_difference_relative', 'number_,_difference',
       'number_,_difference_relative', 'number_-_difference_normalized',
       'number_._difference_relative', 'number_/_difference_normalized',
       'number_NOUN_difference', 'number_[_difference_normalized',
       'number_[_difference_relative', 'number_]_difference',
       'number_]_difference_normalized', 'number_]_difference_relative',
       'number_chara

#### drop the target label and the indexes for training and testing

In [10]:
target_train=feature_dataframe['Translation']
target_test=feature_retrieval['Translation']

In [11]:
target_train

0         1
1         1
2         1
3         1
4         1
         ..
219995    0
219996    0
219997    0
219998    0
219999    0
Name: Translation, Length: 220000, dtype: int64

In [16]:
#scale data into [0,1]
scaler = preprocessing.StandardScaler()
data_train[data_train.columns] = scaler.fit_transform(data_train[data_train.columns])
data_test[data_test.columns] = scaler.transform(data_test[data_test.columns])

# Naive Bayes

In [23]:
nb = GaussianNB().fit(data_train, target_train)
prediction = nb.predict_proba(data_test)
print("The MAP score on test set: {:.4f}".format(MAP_score(feature_retrieval['source_id'],target_test,prediction)))
# acc = accuracy_score(target_test,prediction) 
# f1= f1_score(target_test,prediction) 
# pr= precision_score(target_test,prediction) 
# re= recall_score(target_test,prediction)
# ll=log_loss(target_test,prediction)
# print("The Accuracy on test set: {:.4f}".format(acc))
# print("The F1-Score on test set: {:.4f}".format(f1))
# print("The Precision-Score on test set: {:.4f}".format(pr))
# print("The Recall-Score on test set: {:.4f}".format(re))
# print("The Los_loss on test set: {:.4f}".format(ll))

The MAP score on test set: 0.6814


In [24]:
prediction

array([[1.32887645e-13, 1.00000000e+00],
       [9.96550590e-01, 3.44941027e-03],
       [9.95816197e-01, 4.18380322e-03],
       ...,
       [9.95318774e-01, 4.68122604e-03],
       [9.92603127e-01, 7.39687276e-03],
       [8.38626184e-01, 1.61373816e-01]])

# MLP Classifier

In [17]:
mlp = MLPClassifier(hidden_layer_sizes=30).fit(data_train, target_train)
prediction = mlp.predict_proba(data_test)
print("The MAP score on test set: {:.4f}".format(MAP_score(feature_retrieval['source_id'],target_test,prediction)))
# acc = accuracy_score(target_test,prediction) 
# f1= f1_score(target_test,prediction) 
# pr= precision_score(target_test,prediction) 
# re= recall_score(target_test,prediction) 
# ll=log_loss(target_test,prediction)
# print("The Accuracy on test set: {:.4f}".format(acc))
# print("The F1-Score on test set: {:.4f}".format(f1))
# print("The Precision-Score on test set: {:.4f}".format(pr))
# print("The Recall-Score on test set: {:.4f}".format(re))
# print("The Los_loss on test set: {:.4f}".format(ll))

The MAP score on test set: 0.7472


In [ ]:
scaler = preprocessing.StandardScaler()
model = MLPClassifier(hidden_layer_sizes=50)
feature_selection(model,scaler,feature_dataframe,feature_retrieval,keep_columns,features)

In [ ]:
result = pd.DataFrame()
result['source_id'] = feature_retrieval['source_id']
result['Translation'] = target_test
result['probabilities'] = [x[1] for x in prediction]
# rank by the source_id and get the ranking for each of the queries for all the documents
result['rank'] = result.groupby('source_id')['probabilities'].rank(method='min', ascending=False)

In [36]:
result

,source_id,Translation,probabilities,rank
0,20000,1,0.999376,1.0
1,20000,0,0.002843,3575.0
2,20000,0,0.014845,1134.0
3,20000,0,0.020342,875.0
4,20000,0,0.007698,1899.0
...,...,...,...,...
499995,20099,0,0.018376,3407.0
499996,20099,0,0.018596,3341.0
499997,20099,0,0.015554,4299.0
499998,20099,0,0.048236,1119.0


# Logistic Regression

In [31]:
lr = LogisticRegression(max_iter=100000).fit(data_train, target_train)
prediction = lr.predict_proba(data_test)
print("The MAP score on test set: {:.4f}".format(MAP_score(feature_retrieval['source_id'],target_test,prediction)))
# acc = accuracy_score(target_test,prediction) 
# f1= f1_score(target_test,prediction) 
# pr= precision_score(target_test,prediction) 
# re= recall_score(target_test,prediction) 
# ll=log_loss(target_test,prediction)
# print("The Accuracy on test set: {:.4f}".format(acc))
# print("The F1-Score on test set: {:.4f}".format(f1))
# print("The Precision-Score on test set: {:.4f}".format(pr))
# print("The Recall-Score on test set: {:.4f}".format(re))
# print("The Los_loss on test set: {:.4f}".format(ll))

The MAP score on test set: 0.7458


In [6]:
scaler = preprocessing.StandardScaler()
model = LogisticRegression(max_iter=100000)
feature_selection(model,scaler,feature_dataframe,feature_retrieval,keep_columns,features)

The initial MAP score on test set: 0.7458
With number_punctuations_total_difference added, the MAP score on test set: 0.7268
With number_punctuations_total_difference_relative added, the MAP score on test set: 0.7403
With number_punctuations_total_difference_normalized added, the MAP score on test set: 0.7452
With number_words_difference added, the MAP score on test set: 0.6551
With number_words_difference_relative added, the MAP score on test set: 0.6027
With number_!_difference added, the MAP score on test set: 0.7462
Updated MAP score on test set with new feature number_!_difference: 0.7462
With number_!_difference_relative added, the MAP score on test set: 0.7461
With number_!_difference_normalized added, the MAP score on test set: 0.7150
With number_"_difference added, the MAP score on test set: 0.7462
With number_"_difference_relative added, the MAP score on test set: 0.7462
With number_"_difference_normalized added, the MAP score on test set: 0.7462
With number_#_difference adde

With number__difference_normalized added, the MAP score on test set: 0.7541
With score_polarity_difference added, the MAP score on test set: 0.7490
With score_polarity_difference_relative added, the MAP score on test set: 0.7541
Updated MAP score on test set with new feature score_polarity_difference_relative: 0.7541
With score_polarity_difference_normalized added, the MAP score on test set: 0.7491
With score_subjectivity_difference added, the MAP score on test set: 0.7489
With score_subjectivity_difference_relative added, the MAP score on test set: 0.7541
With score_subjectivity_difference_normalized added, the MAP score on test set: 0.7541
With number_stopwords_difference_relative added, the MAP score on test set: 0.6759
With number_stopwords_difference_normalized added, the MAP score on test set: 0.7541


In [7]:
keep_columns

['cosine_similarity_average',
 'euclidean_distance_average',
 'jaccard_translation_source',
 'jaccard_numbers_source',
 'pca_embeddding_average_diff_0',
 'pca_embeddding_tf_idf_diff_0',
 'pca_embeddding_average_diff_1',
 'pca_embeddding_average_diff_2',
 'pca_embeddding_tf_idf_diff_2',
 'pca_embeddding_average_diff_3',
 'pca_embeddding_average_diff_4',
 'pca_embeddding_average_diff_5',
 'pca_embeddding_average_diff_6',
 'pca_embeddding_average_diff_7',
 'pca_embeddding_average_diff_8',
 'pca_embeddding_average_diff_9',
 'number_!_difference',
 'number_$_difference',
 'number_%_difference',
 'number_%_difference_relative',
 'number_&_difference_relative',
 'number_-_difference_relative',
 'number_:_difference',
 'number_:_difference_relative',
 'number_;_difference',
 'number_;_difference_relative',
 'number_;_difference_normalized',
 'number_[_difference',
 'number_VERB_difference_normalized',
 'score_polarity_difference_relative']

In [32]:
result = pd.DataFrame()
result['source_id'] = feature_retrieval['source_id']
result['Translation'] = target_test
result['probabilities'] = [x[1] for x in prediction]
# rank by the source_id and get the ranking for each of the queries for all the documents
result['rank'] = result.groupby('source_id')['probabilities'].rank(method='max', ascending=False)

In [33]:
result

,source_id,Translation,probabilities,rank
0,20000,1,0.986928,1.0
1,20000,0,0.006586,2594.0
2,20000,0,0.005216,4372.0
3,20000,0,0.005355,4274.0
4,20000,0,0.005774,3912.0
...,...,...,...,...
499995,20099,0,0.005452,1016.0
499996,20099,0,0.005629,588.0
499997,20099,0,0.005241,1977.0
499998,20099,0,0.024840,429.0
